In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/laura/.secret/yelp_api.json', 'r') as f: 
    login = json.load(f)

In [3]:
with open('/Users/laura/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)

In [4]:
yelp = YelpAPI(login['API Key'], timeout_s=5.0)

In [5]:
Location = 'Sioux Falls, SD'
Term = 'Chicken'

In [6]:
JSON_FILE = "Data/results_of_chicken_SF.json" 
JSON_FILE

'Data/results_of_chicken_SF.json'

In [7]:
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_of_chicken_SF.json not found. Saving empty list to file.


In [8]:
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


In [10]:
results = yelp.search_query(location=Location,
                                term=Term,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [11]:
total_results = results['total']
total_results

371

In [12]:
results_per_page = len(results['businesses'])
results_per_page


20

In [13]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

19

In [14]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [20]:
for i in tqdm_notebook(range(1,n_pages+1)): 
    with open(JSON_FILE, 'r') as f: 
        previous_results = json.load(f)
    
    n_results = len(previous_results) 
    
    if (n_results + results_per_page) > 1000: 
        print('Exceed 1000 api calls') 
        break
    results = yelp.search_query(location= Location,
                               term=Term,
                               offset=n_results)
    previous_results.extend(results['businesses'])
    
    with open(JSON_FILE, 'w') as f: 
        json.dump(previous_results, f) 
    
    time.sleep(.2)
    

  0%|          | 0/19 [00:00<?, ?it/s]

In [21]:
df = pd.read_json(JSON_FILE)
df.head(5)

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,9P3l1Sv3mqHPjqDlgXtrfA,oppa-chicken-sioux-falls,Oppa Chicken,https://s3-media1.fl.yelpcdn.com/bphoto/ePqHHh...,False,https://www.yelp.com/biz/oppa-chicken-sioux-fa...,13,"[{'alias': 'korean', 'title': 'Korean'}, {'ali...",4.5,"{'latitude': 43.513924, 'longitude': -96.750413}","[delivery, pickup]","{'address1': '3412 S Western Ave', 'address2':...",+16053348888,(605) 334-8888,3653.340099,NaN
1,VyLu3LAuw1P-nYTQdtjFiA,the-keg-sioux-falls-3,The Keg,https://s3-media4.fl.yelpcdn.com/bphoto/Bdxyis...,False,https://www.yelp.com/biz/the-keg-sioux-falls-3...,35,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 43.5437927, 'longitude': -96.7761...","[delivery, pickup]","{'address1': '4211 W 12th St', 'address2': '',...",+16052754534,(605) 275-4534,4264.988878,$
2,CbWRnnOkmG03AG034bNh6A,slim-chickens-sioux-falls,Slim Chickens,https://s3-media1.fl.yelpcdn.com/bphoto/rYucZo...,False,https://www.yelp.com/biz/slim-chickens-sioux-f...,38,"[{'alias': 'chicken_wings', 'title': 'Chicken ...",3.5,"{'latitude': 43.53146, 'longitude': -96.7317}","[delivery, pickup]","{'address1': '1517 S Minnesota Ave', 'address2...",+16052715227,(605) 271-5227,1166.145206,$
3,ZnpTSWE4InWdXGcy5KlB2g,botskis-sioux-falls,Botski's,https://s3-media1.fl.yelpcdn.com/bphoto/Zb4Ecz...,False,https://www.yelp.com/biz/botskis-sioux-falls?a...,23,"[{'alias': 'tradamerican', 'title': 'American ...",4.0,"{'latitude': 43.5276286306062, 'longitude': -9...",[delivery],"{'address1': '1914 S Sycamore Ave', 'address2'...",+16052715353,(605) 271-5353,4649.274662,$$
4,hJqOZ8CiYlkwdi2oQl571A,swamp-daddys-cajun-kitchen-sioux-falls,Swamp Daddy's Cajun Kitchen,https://s3-media4.fl.yelpcdn.com/bphoto/W5dn1y...,False,https://www.yelp.com/biz/swamp-daddys-cajun-ki...,46,"[{'alias': 'cajun', 'title': 'Cajun/Creole'}, ...",4.0,"{'latitude': 43.55192359999999, 'longitude': -...","[delivery, pickup]","{'address1': '421 North Phillips Ave', 'addres...",+16055951961,(605) 595-1961,1314.305217,NaN


In [23]:
df.tail()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
357,Cm9aL7CgNhc6ssrQQf8Kvg,olive-garden-italian-restaurant-sioux-falls-3,Olive Garden Italian Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/Q3iuMj...,False,https://www.yelp.com/biz/olive-garden-italian-...,65,"[{'alias': 'italian', 'title': 'Italian'}, {'a...",2.5,"{'latitude': 43.514061, 'longitude': -96.76485}",[],"{'address1': '3121 W 41st St', 'address2': '',...",+16053330290,(605) 333-0290,4429.427063,$$
358,DP-JrMQDFnZadPuNMlS_Ww,pizza-hut-sioux-falls-16,Pizza Hut,https://s3-media3.fl.yelpcdn.com/bphoto/s4rm_R...,False,https://www.yelp.com/biz/pizza-hut-sioux-falls...,18,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 43.514388, 'longitude': -96.807279}","[pickup, delivery]","{'address1': '6507 W. 41st Street', 'address2'...",+16053616666,(605) 361-6666,7331.486858,$
359,1VIJ-7F1bs7c-M1RlPvKmQ,dominos-pizza-sioux-falls-4,Domino's Pizza,,False,https://www.yelp.com/biz/dominos-pizza-sioux-f...,9,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 43.5443954467773, 'longitude': -9...",[],"{'address1': '4009 E 10th St', 'address2': Non...",+16053315171,(605) 331-5171,3882.265907,$
360,c76uYNJNsO5i8g2x8piGDg,dominos-pizza-sioux-falls-6,Domino's Pizza,https://s3-media3.fl.yelpcdn.com/bphoto/X2DA21...,False,https://www.yelp.com/biz/dominos-pizza-sioux-f...,10,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.5,"{'latitude': 43.5002151, 'longitude': -96.712106}",[],"{'address1': '1207 E 57th St', 'address2': Non...",+16053713322,(605) 371-3322,4660.154988,$
361,6Fp8rHs3j0vd5RyDx1XKEg,pizza-hut-brandon-11,Pizza Hut,https://s3-media3.fl.yelpcdn.com/bphoto/xyvR7P...,False,https://www.yelp.com/biz/pizza-hut-brandon-11?...,5,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",2.0,"{'latitude': 43.59513651987468, 'longitude': -...","[pickup, delivery]","{'address1': '101 N Splitrock Blvd', 'address2...",+16055824488,(605) 582-4488,13577.648810,$


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 362 entries, 0 to 361
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             362 non-null    object 
 1   alias          362 non-null    object 
 2   name           362 non-null    object 
 3   image_url      362 non-null    object 
 4   is_closed      362 non-null    bool   
 5   url            362 non-null    object 
 6   review_count   362 non-null    int64  
 7   categories     362 non-null    object 
 8   rating         362 non-null    float64
 9   coordinates    362 non-null    object 
 10  transactions   362 non-null    object 
 11  location       362 non-null    object 
 12  phone          362 non-null    object 
 13  display_phone  362 non-null    object 
 14  distance       362 non-null    float64
 15  price          250 non-null    object 
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 42.9+ KB


In [25]:
df.duplicated(subset='id').sum()

0

In [26]:
df.to_csv("Data/results_of_chicken_SF.json", compression='gzip', index=False)